In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta


supermarkets = ['Naivas', 'Quickmart', 'Carrefour', 'Tuskys', 'Chandarana', 'Uchumi']
mpesa_agents = ['MPesa Agent 25401', 'MPesa Agent 25402', 'MPesa Agent 25403']

data = []
start_date = datetime(2025, 4, 1)
transaction_types = ['Buy Goods', 'Withdrawal', 'Deposit', 'Send Money', 'Paybill']

for i in range(50):
    date = start_date + timedelta(days=random.randint(0, 59), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    data.append({
        'transaction_id': f'MP{random.randint(100000, 999999)}',
        'customer_phone': f'+2547{random.randint(10_000_000, 99_999_999)}',
        'supermarket': random.choice(supermarkets),
        'agent': random.choice(mpesa_agents),
        'transaction_type': random.choice(transaction_types),
        'amount': random.randint(100, 10000),
        'date': date.strftime('%Y-%m-%d'),
        'last_updated': date.strftime('%Y-%m-%d %H:%M:%S')
    })

df = pd.DataFrame(data)
df.to_csv('custom_data.csv', index=False)
df.head()

,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
0,MP582479,+254754921293,Quickmart,MPesa Agent 25401,Paybill,4744,2025-05-22,2025-05-22 09:46:00
1,MP278093,+254762266008,Tuskys,MPesa Agent 25401,Deposit,4957,2025-04-22,2025-04-22 11:27:00
2,MP780475,+254763869401,Naivas,MPesa Agent 25402,Deposit,4295,2025-05-01,2025-05-01 19:32:00
3,MP399427,+254734790438,Quickmart,MPesa Agent 25402,Withdrawal,7081,2025-05-16,2025-05-16 20:47:00
4,MP885517,+254741923334,Chandarana,MPesa Agent 25401,Paybill,9916,2025-04-28,2025-04-28 22:48:00


In [4]:
df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print(f"Extracted {len(df_full)} rows fully.")
print(f"Columns: {list(df_full.columns)}")
df_full.head()

Extracted 50 rows fully.
Columns: ['transaction_id', 'customer_phone', 'supermarket', 'agent', 'transaction_type', 'amount', 'date', 'last_updated']


,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
0,MP582479,254754921293,Quickmart,MPesa Agent 25401,Paybill,4744,2025-05-22,2025-05-22 09:46:00
1,MP278093,254762266008,Tuskys,MPesa Agent 25401,Deposit,4957,2025-04-22,2025-04-22 11:27:00
2,MP780475,254763869401,Naivas,MPesa Agent 25402,Deposit,4295,2025-05-01,2025-05-01 19:32:00
3,MP399427,254734790438,Quickmart,MPesa Agent 25402,Withdrawal,7081,2025-05-16,2025-05-16 20:47:00
4,MP885517,254741923334,Chandarana,MPesa Agent 25401,Paybill,9916,2025-04-28,2025-04-28 22:48:00


In [5]:
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-15 08:00:00")

In [6]:
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Extracted {len(df_incremental)} rows incrementally since {last_extraction}.")
df_incremental.head()

Extracted 44 rows incrementally since 2025-04-15 08:00:00.


,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
0,MP582479,254754921293,Quickmart,MPesa Agent 25401,Paybill,4744,2025-05-22,2025-05-22 09:46:00
1,MP278093,254762266008,Tuskys,MPesa Agent 25401,Deposit,4957,2025-04-22,2025-04-22 11:27:00
2,MP780475,254763869401,Naivas,MPesa Agent 25402,Deposit,4295,2025-05-01,2025-05-01 19:32:00
3,MP399427,254734790438,Quickmart,MPesa Agent 25402,Withdrawal,7081,2025-05-16,2025-05-16 20:47:00
4,MP885517,254741923334,Chandarana,MPesa Agent 25401,Paybill,9916,2025-04-28,2025-04-28 22:48:00


In [7]:
if not df.empty:
    new_checkpoint = df['last_updated'].max()
    with open("last_extraction.txt", "w") as f:
        f.write(str(new_checkpoint))
    print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-05-29 18:29:00


In [ ]:
# Tranform full data 
df_full_clean = df_full.drop_duplicates()

df_full_clean['transaction_fee'] = df_full_clean['amount'] * 0.015

df_full_clean['date'] = pd.to_datetime(df_full_clean['date']).dt.strftime('%Y-%m-%d')

df_full_clean['amount_category'] = pd.cut(df_full_clean['amount'],
                                          bins=[0, 1000, 5000, 10000],
                                          labels=['Small', 'Medium', 'Large'])

df_full_clean.to_csv('transformed_full.csv', index=False)
df_full_clean.head()

,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated,transaction_fee,amount_category
0,MP582479,254754921293,Quickmart,MPesa Agent 25401,Paybill,4744,2025-05-22,2025-05-22 09:46:00,71.160,Medium
1,MP278093,254762266008,Tuskys,MPesa Agent 25401,Deposit,4957,2025-04-22,2025-04-22 11:27:00,74.355,Medium
2,MP780475,254763869401,Naivas,MPesa Agent 25402,Deposit,4295,2025-05-01,2025-05-01 19:32:00,64.425,Medium
3,MP399427,254734790438,Quickmart,MPesa Agent 25402,Withdrawal,7081,2025-05-16,2025-05-16 20:47:00,106.215,Large
4,MP885517,254741923334,Chandarana,MPesa Agent 25401,Paybill,9916,2025-04-28,2025-04-28 22:48:00,148.740,Large


In [9]:
# Transform incremental data
df_incremental_clean = df_incremental.drop_duplicates()

df_incremental_clean['transaction_fee'] = df_incremental_clean['amount'] * 0.015

df_incremental_clean['date'] = pd.to_datetime(df_incremental_clean['date']).dt.strftime('%Y-%m-%d')

df_incremental_clean['amount_category'] = pd.cut(df_incremental_clean['amount'],
                                                 bins=[0, 1000, 5000, 10000],
                                                 labels=['Small', 'Medium', 'Large'])

df_incremental_clean.to_csv('transformed_incremental.csv', index=False)
df_incremental_clean.head()

,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated,transaction_fee,amount_category
0,MP582479,254754921293,Quickmart,MPesa Agent 25401,Paybill,4744,2025-05-22,2025-05-22 09:46:00,71.160,Medium
1,MP278093,254762266008,Tuskys,MPesa Agent 25401,Deposit,4957,2025-04-22,2025-04-22 11:27:00,74.355,Medium
2,MP780475,254763869401,Naivas,MPesa Agent 25402,Deposit,4295,2025-05-01,2025-05-01 19:32:00,64.425,Medium
3,MP399427,254734790438,Quickmart,MPesa Agent 25402,Withdrawal,7081,2025-05-16,2025-05-16 20:47:00,106.215,Large
4,MP885517,254741923334,Chandarana,MPesa Agent 25401,Paybill,9916,2025-04-28,2025-04-28 22:48:00,148.740,Large
